In [3]:
import pandas as pd
import json

# Load the JSON file
file_path = "results/previous_dataset.json"  # Update this if needed
df = pd.read_json(file_path)

# Remove only the 'book_source' column if it exists
if 'book_source' in df.columns:
    df.drop(columns=['book_source'], inplace=True)

# Save the modified JSON file
output_path = "modified_data.json"
df.to_json(output_path, orient="records", indent=4)

print(f"Modified JSON saved at: {output_path}")


Modified JSON saved at: modified_data.json


In [2]:
import pandas as pd
import json

# Load the JSON file
file_path = "results/labeled_interview_qa.json"  # Update this path if needed
df = pd.read_json(file_path)

# Remove entries where "labeled_topic" is 63 or 23
df_filtered = df[~df["labeled_topic"].isin([63, 23])]

# Save the modified JSON file
output_path = "results/filtered_interview_qa.json"
df_filtered.to_json(output_path, orient="records", indent=4)

print(f"Filtered JSON saved at: {output_path}")


Filtered JSON saved at: results/filtered_interview_qa.json


In [3]:
import pandas as pd
from bertopic import BERTopic

# Load your dataset from the JSON file
df = pd.read_json('results/raw_interview_qa.json')


# merge the question and answer columns in one text
texts_subset = df['question'] + " " + df['answer']

# Create a BERTopic model instance (using default parameters for simplicity)
topic_model = BERTopic(language="english", calculate_probabilities=True)

# Fit the model to your texts and get topics (ignore probabilities for now)
topics_subset, _ = topic_model.fit_transform(texts_subset)

# Add a new column 'labeled_topic' to the subset DataFrame with the predicted topics
df['labeled_topic'] = topics_subset

# Print the subset with the new 'labeled_topic' column to inspect the results
print(df[['answer', 'labeled_topic']])

# Optionally, save the labeled subset to a new JSON file
output_path = 'results/labeled_interview_qa_bertopic.json'
df.to_json(output_path, orient='records', indent=4)
print(f"Labeled data for the first 10 entries saved to {output_path}")


2025-02-23 03:46:09.375115: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 03:46:09.396596: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-23 03:46:09.546394: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-23 03:46:09.547239: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-23 03:46:10.340088: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

                                                  answer  labeled_topic
0      Generative AI can significantly enhance threat...             -1
1      Teams developing generative AI models have emp...             -1
2      Generative AI developers use techniques like k...            329
3      Common pitfalls include data bias and heteroge...             57
4      Techniques such as gradient-based explanations...             -1
...                                                  ...            ...
28034  Ji-Rong Wen is a full professor, the executive...            383
28035  Yankai Lin's main research interests are pretr...            383
28036  Yankai Lin is interested in pretrained models ...            383
28037  Prior to his current role at Renmin University...            383
28038  Yankai Lin is currently an assistant professor...            383

[28039 rows x 2 columns]
Labeled data for the first 10 entries saved to results/labeled_interview_qa_bertopic.json


In [5]:
# Extract topic information
topic_info = topic_model.get_topic_info()

# Initialize a dictionary to hold all topic details
all_topics = {}

# Iterate over each topic to get its details
for topic_id in topic_info['Topic']:
    if topic_id == -1:
        # Skip the outlier topic
        continue
    # Get the top words for the topic
    words = topic_model.get_topic(topic_id)
    # Get the size of the topic
    size = topic_info[topic_info['Topic'] == topic_id]['Count'].values[0]
    # Store the information in the dictionary
    all_topics[topic_id] = {
        'size': size,
        'words': words
    }
    print(f"Topic {topic_id}: {size} documents")
    print(f"Words: {words}")

# Save the topics and their details to a JSON file


# # Save the topics and their details to a JSON file
# with open('results/topic_details.json', 'w') as f:
#     json.dump(all_topics, f, indent=4)

# # Optionally, save the BERTopic model for future use
# # Method 1 - Using safetensors (recommended for smaller size and safety)
# embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

# # Method 2 - Using pickle (saves the full model but results in a larger file)
# # topic_model.save("results/bertopic_model.pkl", serialization="pickle")

Topic 0: 277 documents
Words: [('tokenization', 0.037006709772957516), ('tokenizer', 0.032433852452414404), ('tokens', 0.018799932563503947), ('subword', 0.01687064083196457), ('tokenizers', 0.014871788637658508), ('ids', 0.013185950961998686), ('special', 0.013050814631867603), ('subwords', 0.01197821623874519), ('bpe', 0.011329387712829759), ('wordpiece', 0.010870682954780586)]
Topic 1: 219 documents
Words: [('multimodal', 0.0286649062111254), ('modalities', 0.01406662815731792), ('generative', 0.01345544010261238), ('language', 0.011332533118149735), ('audio', 0.010832714070261606), ('types', 0.009973354083035534), ('primarily', 0.00996172277424483), ('images', 0.009868625075655703), ('generate', 0.00952957693314969), ('differ', 0.009434629482151353)]
Topic 2: 193 documents
Words: [('matplotlib', 0.03743373937086404), ('plot', 0.029062293559105813), ('plots', 0.029009787572692354), ('visualization', 0.01694544434485867), ('color', 0.016098576934978955), ('scatter', 0.015159821507391

In [9]:
# Extract topic information
topic_info = topic_model.get_topic_info()

# Prepare a dictionary for all topics with native types
all_topics = {}

for topic_id in topic_info['Topic']:
    if topic_id == -1:  # Skip outliers
        continue
    python_topic_id = int(topic_id)  # Convert topic ID to native int
    words = topic_model.get_topic(python_topic_id)
    size = topic_info[topic_info['Topic'] == topic_id]['Count'].values[0]
    python_size = int(size)  # Convert count to native int

    # Convert words (list of tuples) to a list of lists with native types
    converted_words = []
    for word, weight in words:
        converted_words.append([word, float(weight)])
    
    all_topics[python_topic_id] = {
        'size': python_size,
        'words': converted_words
    }
    
    print(f"Topic {python_topic_id}: {python_size} documents")
    print(f"Words: {converted_words}")

# Save the topic details to a JSON file
output_topics = 'results/topic_details.json'
with open(output_topics, 'w') as f:
    json.dump(all_topics, f, indent=4)
print(f"Topic details saved to {output_topics}")

Topic 0: 277 documents
Words: [['tokenization', 0.037006709772957516], ['tokenizer', 0.032433852452414404], ['tokens', 0.018799932563503947], ['subword', 0.01687064083196457], ['tokenizers', 0.014871788637658508], ['ids', 0.013185950961998686], ['special', 0.013050814631867603], ['subwords', 0.01197821623874519], ['bpe', 0.011329387712829759], ['wordpiece', 0.010870682954780586]]
Topic 1: 219 documents
Words: [['multimodal', 0.0286649062111254], ['modalities', 0.01406662815731792], ['generative', 0.01345544010261238], ['language', 0.011332533118149735], ['audio', 0.010832714070261606], ['types', 0.009973354083035534], ['primarily', 0.00996172277424483], ['images', 0.009868625075655703], ['generate', 0.00952957693314969], ['differ', 0.009434629482151353]]
Topic 2: 193 documents
Words: [['matplotlib', 0.03743373937086404], ['plot', 0.029062293559105813], ['plots', 0.029009787572692354], ['visualization', 0.01694544434485867], ['color', 0.016098576934978955], ['scatter', 0.015159821507391